Removal of covariance for variables screened by MLR and RFECV  
MLR, VPM and MSEM in R-code

对MLR和RFECV筛选的变量去除共线性  
MLR,VPM和MSEM在R代码

In [1]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import myrfecv as mr

path_data_raw = "C:/Users/dell/OneDrive/file/"
path_country_nc = "C:/Users/dell/OneDrive/file/nc"
path_onedrive_csv = "C:/Users/dell/OneDrive/file/csv/"
path_one_spdb = 'C:/Users/dell/OneDrive/file/SPDB/'
drive_letter = 'E:'

path_pre = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/pretreatment/"
path_match = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/match/"
path_semdata = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/semdata/"

path_2_preanalysis_data = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part2_sem/preanalysis/"
path_2_preanalysis_fig = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part2_sem/preanalysis/"
path_3_sw_forecast = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/"
path_temp = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/temp/"


mark_num = "25"
meta_name = "meta_data.csv"
list_pfas =['PFOA', 'PFNA', 'PFDA', 'PFUnDA','PFDoDA','PFTrDA', 'PFTeDA', 'PFHxS', 'PFOS', 'FOSA', 'PFBA', 'PFPeA', 'PFHxA', 'PFHpA','PFBS']
list_pfas_lc = ['PFOA', 'PFNA', 'PFDA', 'PFUnDA','PFDoDA','PFTrDA', 'PFTeDA', 'PFHxS', 'PFOS', 'FOSA']
list_pfas_sc = ['PFBA', 'PFPeA', 'PFHxA', 'PFHpA','PFBS']
list_color = ["#4d8cbf", "#4f9c8b", "#555c6c", "#d77563", "#7d84a8", "#84aeb8", "#c3473b", "#89756d","#ffb3cc","#9a7ebf","#ffddb8", "#c4eaff", "#d1c6ff", "#c2ffbf", "#f5f5b0"]
dic_color = dict(zip(list_pfas,list_color))

In [ ]:
from factor_analyzer import FactorAnalyzer
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats

def count_vif(df_features_raw, max_vif=10):
    """
    Iteratively calculate the input dataframe VIF removing the variables with the largest VIFs with each iteration until the VIFs of all variables are less than 10.
    Returns a dataframe containing two columns of variables VIF
    ---
    迭代计算输入的dataframe VIF 每次迭代去除VIF最大的变量 直到所有变量的VIF小于10
    返回一个dataframe 里面包含两列 variables VIF
    """
    vif_data = pd.DataFrame()
    vif_data["variables"] = df_features_raw.columns
    while True:
        # 计算VIF
        vif_data["VIF"] = [variance_inflation_factor(df_features_raw.values, i) 
                        for i in range(len(df_features_raw.columns))]
        
        if vif_data['VIF'].max() > max_vif:
            max_vif_feature = vif_data.loc[vif_data['VIF'].idxmax(), 'variables']
            df_features_raw = df_features_raw.drop(max_vif_feature, axis=1)
            vif_data = vif_data[vif_data['variables'] != max_vif_feature]
            print('Remove var:',max_vif_feature)
        else:
            break
    print(vif_data.shape)
    return vif_data

In [ ]:
path_sw_rfecv_data = drive_letter + '/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/'
path_lrsw_rfecv_data = drive_letter + '/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/lrsw_forecast/'

str_describe = 'sws7'
sw_mark = 's7'

str_describe = 'sws7'
df_mlr = pd.read_csv(path_2_preanalysis_data + 'mlr_'+str_describe+'.csv')
df_mlr = df_mlr.sort_values(by='R2', ascending=False)
df_rfecv = pd.read_csv(path_sw_rfecv_data + str_describe + '_merge_rfecv.csv')
min_features = df_rfecv.loc[0, 'min_features']
max_model = df_rfecv.loc[0, 'model']
max_score = round(df_rfecv.loc[0, 'mean_test_score'],3)
print(f'model:{max_model} min_feature:{min_features} score:{max_score}')
df_ml  =pd.read_csv(path_sw_rfecv_data + str_describe + '_rfecv_features_'+max_model+'cv.csv')
df_ml = df_ml[df_ml['Rank']==1]
list_mlr_var0 = df_mlr['variables'].values
list_ml_var0 = df_ml['Feature'].values
list_re_imp0 = list(set((set(list_mlr_var0) | set(list_ml_var0))))


df_data = pd.read_csv(path_semdata + 'sem_sw_' + sw_mark + '_au_to_'+mark_num+'_avg.csv')

df_data_vif = df_data[list_re_imp0]
df_vif_sfs = count_vif(df_data_vif)

df_vif_sfs.to_csv(path_2_preanalysis_data +'vif_sfs_'+str_describe+'_10_3_r.csv', index=False)

In [ ]:
str_describe = 'lr_sws7'
sw_mark = 'lr_sws7'

str_describe = 'sws7'
df_mlr = pd.read_csv(path_2_preanalysis_data + 'mlr_'+str_describe+'.csv')
df_mlr = df_mlr.sort_values(by='R2', ascending=False)
df_rfecv = pd.read_csv(path_lrsw_rfecv_data + str_describe + '_merge_rfecv.csv')
min_features = df_rfecv.loc[0, 'min_features']
max_model = df_rfecv.loc[0, 'model']
max_score = round(df_rfecv.loc[0, 'mean_test_score'],3)
print(f'model:{max_model} min_feature:{min_features} score:{max_score}')
df_ml  =pd.read_csv(path_lrsw_rfecv_data + str_describe + '_rfecv_features_'+max_model+'cv.csv')
df_ml = df_ml[df_ml['Rank']==1]
list_mlr_var0 = df_mlr['variables'].values
list_ml_var0 = df_ml['Feature'].values
list_re_imp0 = list(set((set(list_mlr_var0) | set(list_ml_var0))))
df_data = pd.read_csv(path_semdata + 'sem_' + sw_mark + '_au_to_'+mark_num+'_avg.csv')
df_data_vif = df_data[list_re_imp0]
df_vif_sfs = count_vif(df_data_vif)
df_vif_sfs.to_csv(path_2_preanalysis_data +'vif_sfs_'+str_describe+ '_10_3_r.csv', index=False)

In [ ]:
# View variables
# 查看变量
str_describe = 'lr_sws7'
sw_mark = 'sws7'
df_vif_lr = pd.read_csv(path_2_preanalysis_data + 'vif_sfs_lr_sws7_10_3_r.csv')
df_vif_sw = pd.read_csv(path_2_preanalysis_data + 'vif_sfs_sws7_10_3_r.csv')

list_vars_lr = df_vif_lr['variables'].tolist()
list_vars_sw = df_vif_sw['variables'].tolist()


print('lr_all:', len(list_vars_lr))
print('sw_all:', len(list_vars_sw))

meta_data = pd.read_csv("C:\\Users\\dell\\OneDrive\\file\\meta_data.csv")

list_hue_var = meta_data['var_name'][meta_data['var_type3']==0].tolist()
list_env_var = meta_data['var_name'][meta_data['var_type3']==1].tolist()
list_pfas_var = meta_data['var_name'][meta_data['var_type3']==2].tolist()
list_sp_var = meta_data['var_name'][meta_data['var_type3']==3].tolist()


list_lr_hue = list(set(list_vars_lr) & set(list_hue_var))
list_lr_env = list(set(list_vars_lr) & set(list_env_var))
list_lr_pfas = list(set(list_vars_lr) & set(list_pfas_var))
list_lr_sp = list(set(list_vars_lr) & set(list_sp_var))

list_sw_hue = list(set(list_vars_sw) & set(list_hue_var))
list_sw_env = list(set(list_vars_sw) & set(list_env_var))
list_sw_pfas = list(set(list_vars_sw) & set(list_pfas_var))


print(f'lr hue:{list_lr_hue}')
print(f'lr env:{list_lr_env}')
print(f'lr pfas:{list_lr_pfas}')
print(f'lr sp:{list_lr_sp}')

print(f'sw hue:{list_sw_hue}')
print(f'sw env:{list_sw_env}')
print(f'sw pfas:{list_sw_pfas}')
